# NOTA: Tratamiento de valores nulos en la base Italia

### Variable `banos`
- **Método:** Extracción numérica desde `texto_banos`. Los casos `"half-bath"` se imputaron como `0.5`. Los `NaN` se reemplazaron con la **moda**.  
- **Justificación:** El texto siempre contiene la cantidad de baños. Para asegurar consistencia, los valores faltantes se imputan con la moda, pues representa el número más común en el dataset.

---

### Variable `camas`
- **Método:** Imputación con la **moda**.  
- **Justificación:** El número de camas suele ser discreto y repetitivo. La moda representa el valor más frecuente y es el mejor sustituto en este caso.

---

### Variable `recamaras`
- **Método:** Imputación con la **mediana**.  
- **Justificación:** Se eligió la mediana porque el número de recámaras puede estar sesgado por valores extremos (ej. propiedades muy grandes). La mediana es más robusta ante outliers.

---

### Variable `texto_banos`
- **Método:** Reemplazo de `NaN` con `"1 bath"`.  
- **Justificación:** Permite mantener consistencia con la imputación realizada en `banos` y evita discrepancias entre ambas variables.

---

### Variable `precio`
- **Método:** Conversión de texto a número (eliminando `$` y comas) e imputación con la **media**.  
- **Justificación:** La media refleja un promedio del mercado, lo cual es razonable como sustituto en este caso donde el precio puede ser continuo.

---

### Variables relacionadas con reseñas  
(`primera_resena`, `ultima_resena`, `resenas_por_mes`, `calif_total`, `calif_exactitud`, `calif_limpieza`, `calif_checkin`, `calif_comunicacion`, `calif_ubicacion`, `calif_valor`)  
- **Método:**  
  - Fechas → `NaN` reemplazados por la fecha sentinela **`01/01/2222`**.  
  - Numéricas → `NaN` reemplazados por **`-222`**.  
- **Justificación:** Estas variables están relacionadas: si no hay reseñas, tampoco hay fechas ni calificaciones. Usar sentinelas numéricos y de fecha permite mantener formato de columna sin perder la señal de “sin reseña”.

---

### Variable `es_superhost`
- **Método:** `t` → `1`, `f` → `0`, y `NaN` → `"SIN_INFO"`.  
- **Justificación:** Permite identificar claramente si un host es superhost, no lo es, o no hay información disponible.

---

### Variable `tiempo_respuesta_host`
- **Método:** `NaN` reemplazados por `"SIN_INFO"`.  
- **Justificación:** Mantiene la columna como categórica y marca explícitamente la ausencia de datos en vez de asumir un valor arbitrario.

---

### Variables de porcentaje (`tasa_respuesta_host`, `tasa_aceptacion_host`)
- **Método:** Conversión de `%` a valores entre 0 y 1. Los `NaN` se reemplazaron por **`222`** como código especial.  
- **Justificación:** La escala 0–1 es estándar para análisis, y el sentinela `222` evita perder la información de “sin datos”.

---

### Variables con pocos nulos  
(`host_tiene_foto`, `host_verificado`, `listados_host`, `total_listados_host`, `antiguedad_host`)  
- **Método:** Imputación con la **moda**.  
- **Justificación:** Al ser pocos casos, imputar con el valor más frecuente preserva la distribución original y evita sesgos.

---

### Variable `licencia`
- **Método:** Reemplazo de `NaN` con el código especial **`SL222222X2XX2XXXX2`**.  
- **Justificación:** Marca de manera explícita las propiedades sin licencia registrada, manteniendo la columna como texto sin introducir valores arbitrarios.


In [139]:
#Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [140]:
#Carga desde un archivo Excel omitiendo las primeras 4 filas
data = pd.read_csv(r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Valores_Nulos\Actividad 3.3\Italia_filtrado_es.csv")
data.head(5)

,id,id_anfitrion,zona_grupo,vecindario,latitud,longitud,tipo_propiedad,tipo_habitacion,capacidad,banos,...,tasa_aceptacion_host,host_tiene_foto,host_verificado,listados_host,total_listados_host,host_total_casas_completas,host_total_habitaciones_privadas,host_total_habitaciones_compartidas,licencia,amenidades
0,44527,120215,Isole,Cannaregio,45.44569,12.32902,Entire rental unit,Entire home/apt,4,2.0,...,100%,t,t,3.0,7.0,2,0,0,IT027042B4GR0PSZS4,"[""Microwave"", ""Heating"", ""Essentials"", ""Air co..."
1,47383,214390,Isole,San Polo,45.43921,12.33310,Room in bed and breakfast,Hotel room,2,1.0,...,100%,t,t,4.0,4.0,1,0,0,IT027042B4D3D4AUUS,"[""Heating"", ""Essentials"", ""Air conditioning"", ..."
2,49656,226598,Isole,Dorsoduro,45.43549,12.32612,Entire rental unit,Entire home/apt,2,1.0,...,100%,t,t,3.0,3.0,2,0,0,IT027042C2L6NZYNJR,"[""Microwave"", ""Heating"", ""Essentials"", ""Air co..."
3,60304,290452,Isole,Cannaregio,45.44453,12.33300,Private room in bed and breakfast,Private room,2,1.0,...,75%,t,t,2.0,2.0,0,2,0,IT027042B4JG3TJH56,"[""Heating"", ""Essentials"", ""Air conditioning"", ..."
4,61714,289023,Isole,Giudecca,45.42708,12.31823,Private room in bed and breakfast,Private room,2,1.0,...,100%,t,t,2.0,6.0,0,2,0,IT027042C1NT8RRIFS,"[""Heating"", ""Essentials"", ""Pack \u2019n play/T..."


In [141]:
#Exploración inicial de los datos
print("Dimensiones:", data.shape)
print("\nTipos de datos:")
print(data.dtypes)
print("\nValores nulos por columna:")
print(data.isnull().sum())

Dimensiones: (8250, 50)

Tipos de datos:
id                                       int64
id_anfitrion                             int64
zona_grupo                              object
vecindario                              object
latitud                                float64
longitud                               float64
tipo_propiedad                          object
tipo_habitacion                         object
capacidad                                int64
banos                                  float64
recamaras                              float64
camas                                  float64
texto_banos                             object
noches_minimas                           int64
noches_maximas                           int64
min_noches_maximas                       int64
max_noches_maximas                       int64
precio                                  object
disp_30d                                 int64
disp_60d                                 int64
disp_90d           

In [142]:
# Copia del dataset para limpiar
data_limpio = data.copy()

In [143]:
# Mapear variantes de half-bath a 0.5
data_limpio.loc[
    data_limpio["texto_banos"].str.strip().str.lower().isin(
        ["shared half-bath", "private half-bath", "half-bath"]
    ),
    "banos"
] = 0.5

# imputar los NaN restantes de 'banos' con la moda
moda_banos = data_limpio["banos"].mode(dropna=True).iloc[0]
data_limpio["banos"] = data_limpio["banos"].fillna(moda_banos)

# verificación
print("NaN en 'banos' después de imputar:", data_limpio["banos"].isna().sum())
print("Moda usada para 'banos':", moda_banos)


NaN en 'banos' después de imputar: 0
Moda usada para 'banos': 1.0


In [144]:
# Imputar NaN en 'recamaras' con la media
media_rec = data_limpio["recamaras"].median(skipna=True)
data_limpio["recamaras"] = data_limpio["recamaras"].fillna(media_rec)

# Verificación
print("NaN en 'recamaras' después de imputar:", data_limpio["recamaras"].isna().sum())
print("Media usada para 'recamaras':", media_rec)


NaN en 'recamaras' después de imputar: 0
Media usada para 'recamaras': 1.0


In [145]:
# Imputar NaN en 'camas' con la moda
moda_camas = data_limpio["camas"].mode(dropna=True).iloc[0]
data_limpio["camas"] = data_limpio["camas"].fillna(moda_camas)

# Verificación
print("NaN en 'camas' después de imputar:", data_limpio["camas"].isna().sum())
print("Moda usada para 'camas':", moda_camas)

NaN en 'camas' después de imputar: 0
Moda usada para 'camas': 1.0


In [146]:
# Reemplazar NaN en 'texto_banos' por "1 bath"
data_limpio["texto_banos"] = data_limpio["texto_banos"].fillna("1 bath")

# Verificación
print("NaN en 'texto_banos' después de imputar:", data_limpio["texto_banos"].isna().sum())
print("Ejemplos de valores en 'texto_banos':")
print(data_limpio["texto_banos"].unique()[:10])


NaN en 'texto_banos' después de imputar: 0
Ejemplos de valores en 'texto_banos':
['2 baths' '1 private bath' '1 bath' '1 shared bath' '7 shared baths'
 'Shared half-bath' '3 baths' '4 baths' '2.5 baths' '1.5 baths']


In [147]:
# Convertir 'precio' a numérico y reemplazar NaN con la media
data_limpio["precio"] = (
    data_limpio["precio"]
    .replace(r"[\$,]", "", regex=True)   # quita $ y comas
    .astype(float)
)

media_precio = data_limpio["precio"].mean(skipna=True)
data_limpio["precio"] = data_limpio["precio"].fillna(media_precio)

# Verificación
print("NaN en 'precio' después de imputar:", data_limpio["precio"].isna().sum())
print("Media usada para 'precio':", media_precio)


NaN en 'precio' después de imputar: 0
Media usada para 'precio': 228.50778129734195


In [148]:
# Columnas relacionadas con reseñas
cols_fechas = ["primera_resena", "ultima_resena"]
cols_numericas = [
    "resenas_por_mes", "calif_total", "calif_exactitud", "calif_limpieza",
    "calif_checkin", "calif_comunicacion", "calif_ubicacion", "calif_valor"
]

# Sentinelas
SENTINELA_FECHA = pd.Timestamp("2222-01-01")
SENTINELA_NUM = -222

# Reemplazar NAs en fechas por sentinela
for col in cols_fechas:
    data_limpio[col] = pd.to_datetime(data_limpio[col], errors="coerce")
    data_limpio[col] = data_limpio[col].fillna(SENTINELA_FECHA)

# Reemplazar NAs en numéricas por sentinela numérico
for col in cols_numericas:
    data_limpio[col] = data_limpio[col].fillna(SENTINELA_NUM)

# Verificación
print("NAs restantes en columnas de reseñas:")
print(data_limpio[cols_fechas + cols_numericas].isna().sum())


NAs restantes en columnas de reseñas:
primera_resena        0
ultima_resena         0
resenas_por_mes       0
calif_total           0
calif_exactitud       0
calif_limpieza        0
calif_checkin         0
calif_comunicacion    0
calif_ubicacion       0
calif_valor           0
dtype: int64


In [149]:
# Reemplazar NaN en es_superhost por 'SIN_INFO'
data_limpio["es_superhost"] = data_limpio["es_superhost"].fillna("SIN_INFO")

print(data_limpio["es_superhost"].value_counts(dropna=False))


es_superhost
f           4645
t           3300
SIN_INFO     305
Name: count, dtype: int64


In [150]:
# Reemplazar NaN en tiempo_respuesta_host por 'SIN_INFO'
data_limpio["tiempo_respuesta_host"] = data_limpio["tiempo_respuesta_host"].fillna("SIN_INFO")

# Verificación
print(data_limpio["tiempo_respuesta_host"].value_counts(dropna=False))


tiempo_respuesta_host
within an hour        6522
SIN_INFO               894
within a few hours     517
within a day           281
a few days or more      36
Name: count, dtype: int64


In [151]:
# Columnas en porcentaje
cols_pct = ["tasa_respuesta_host", "tasa_aceptacion_host"]

for col in cols_pct:
    # Quitar '%' y convertir a float
    data_limpio[col] = (
        data_limpio[col]
        .astype(str)
        .str.replace("%", "", regex=False)
        .replace("nan", np.nan)   # por si NaN quedó como texto
        .astype(float)
    )
    # Convertir a escala 0-1
    data_limpio[col] = data_limpio[col] / 100.0
    # Reemplazar NaN con 222
    data_limpio[col] = data_limpio[col].fillna(222)

# Verificación
for col in cols_pct:
    print(f"NAs restantes en {col}:", data_limpio[col].isna().sum())


NAs restantes en tasa_respuesta_host: 0
NAs restantes en tasa_aceptacion_host: 0


In [152]:
# Columnas con pocos NAs → imputación con la moda
cols_moda = ["host_tiene_foto", "host_verificado", "listados_host", "total_listados_host", "antiguedad_host"]

for col in cols_moda:
    moda_val = data_limpio[col].mode(dropna=True).iloc[0]
    data_limpio[col] = data_limpio[col].fillna(moda_val)
    print(f"Columna '{col}' imputada con la moda: {moda_val} ✅ | NAs restantes: {data_limpio[col].isna().sum()}")


Columna 'host_tiene_foto' imputada con la moda: t ✅ | NAs restantes: 0
Columna 'host_verificado' imputada con la moda: t ✅ | NAs restantes: 0
Columna 'listados_host' imputada con la moda: 1.0 ✅ | NAs restantes: 0
Columna 'total_listados_host' imputada con la moda: 1.0 ✅ | NAs restantes: 0
Columna 'antiguedad_host' imputada con la moda: 2012-06-14 ✅ | NAs restantes: 0


In [153]:
# Reemplazar NaN en 'licencia' con código especial
data_limpio["licencia"] = data_limpio["licencia"].fillna("SL222222X2XX2XXXX2")

# Verificación
print("NAs en 'licencia' después de imputar:", data_limpio["licencia"].isna().sum())
print("Ejemplo de valores en 'licencia':", data_limpio["licencia"].unique()[:10])


NAs en 'licencia' después de imputar: 0
Ejemplo de valores en 'licencia': ['IT027042B4GR0PSZS4' 'IT027042B4D3D4AUUS' 'IT027042C2L6NZYNJR'
 'IT027042B4JG3TJH56' 'IT027042C1NT8RRIFS' 'IT027042B4XVN5TFC6'
 'IT027042C2LG7GAX8J' 'IT027042B40Z6IVYGS' 'IT027042B4I8XOSCJH'
 'SL222222X2XX2XXXX2']


In [154]:
#Exploración final de los datos faltantes
print("\nValores nulos por columna:")
print(data_limpio.isnull().sum())


Valores nulos por columna:
id                                     0
id_anfitrion                           0
zona_grupo                             0
vecindario                             0
latitud                                0
longitud                               0
tipo_propiedad                         0
tipo_habitacion                        0
capacidad                              0
banos                                  0
recamaras                              0
camas                                  0
texto_banos                            0
noches_minimas                         0
noches_maximas                         0
min_noches_maximas                     0
max_noches_maximas                     0
precio                                 0
disp_30d                               0
disp_60d                               0
disp_90d                               0
disp_365d                              0
num_resenas                            0
num_resenas_12m              

In [155]:
# Exportar CSV final sin NAs
data_limpio.to_csv("italia_filtrado_es_sin_NAN.csv", index=False, encoding="utf-8-sig")

print("Archivo 'italia_filtrado_es_sin_NAN.csv' exportado correctamente ✅")


Archivo 'italia_filtrado_es_sin_NAN.csv' exportado correctamente ✅
